In [1]:
import pandas as pd
import numpy as np
import ast
import skimage.io as io
from skimage import color, measure
from skimage.morphology import erosion, dilation, opening, closing, remove_small_holes, disk
from skimage.transform import resize
from skimage.measure import regionprops_table, regionprops, label
from skimage.segmentation import felzenszwalb
from skimage.exposure import match_histograms
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches
from skimage.segmentation import mark_boundaries

In [2]:
# 函数6：将给定的颜色图添加至灰度图
def gray2color(gray_array, color_map):  
    '''Apply color map to the gray image.'''
    maxi = gray_array.max()
    mini = gray_array.min()
    gray_rescale = np.int_(gray_image * 
                           np.ones(gray_image.shape) / maxi * 255)
    rows, cols = gray_array.shape
    color_array = np.zeros((rows, cols, 3), np.uint8) 

    for i in range(0, rows):
        for j in range(0, cols):
            color_array[i, j] = color_map[gray_rescale[i, j]]  

    #color_image = Image.fromarray(color_array)
    return color_array

In [3]:
# 函数7：测量穗粒对象的长度
def length(regionmask):
    # 加黑边
    r, c = regionmask.shape[0], regionmask.shape[1]
    img_dia = np.zeros((r+2, c+2))
    img_dia[1:r+1, 1:c+1] = regionmask
    
    regions = regionprops(label(img_dia))
    centroid = regions[0].centroid
    
#     # 穗粒图像腐蚀一个像素
#     img_dia = erosion(img_dia, disk(1))
    
    img_ero = erosion(img_dia, disk(1))
    img_edge = np.logical_xor(img_ero, img_dia)
    edge_points = np.argwhere(img_edge == True)
    dist_tmp = [np.power(x,2) + np.power(y,2) for x,y in edge_points - centroid]
    end_pt_1 = edge_points[np.argmax(dist_tmp)]
    dist_tmp = [np.power(x,2) + np.power(y,2) for x,y in edge_points - end_pt_1]
    end_pt_2 = edge_points[np.argmax(dist_tmp)]
    long_axis_length_1 = np.linalg.norm(end_pt_1 - end_pt_2)
    
    img_big = dilation(img_dia, disk(1))
    big_edge = np.logical_xor(img_big, img_dia)
    edge_points = np.argwhere(big_edge == True)
    dist_tmp = [np.power(x,2) + np.power(y,2) for x,y in edge_points - centroid]
    end_pt_1 = edge_points[np.argmax(dist_tmp)]
    dist_tmp = [np.power(x,2) + np.power(y,2) for x,y in edge_points - end_pt_1]
    end_pt_2 = edge_points[np.argmax(dist_tmp)]
    long_axis_length_2 = np.linalg.norm(end_pt_1 - end_pt_2)
    
    return (long_axis_length_1 + long_axis_length_2)/2

In [4]:
# 函数8：自定义取整函数
def round_new(num, threshold=4):
    decimal_n = (num*10)%10
    if decimal_n >= threshold:
        result = np.floor((num)%10 + 1)
    else:
        result = np.floor(num)
    return result

In [5]:
# 加载深度学习穗粒分类模型
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np
from torchvision import models

import torchvision
import torchvision.models as models
import torch.nn as nn
from PIL import Image
model = torchvision.models.vgg16(pretrained=False)
model.classifier._modules['6'] = nn.Linear(4096, 3)
model.load_state_dict(torch.load(r'./Data/model_weights.pth'))
model.cuda()
model.eval()
# 模型输入标定
tran = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()
])

In [6]:
# 提取jet颜色图数据
from matplotlib import cm 
colormap_int = np.zeros((256, 3), np.uint8)
colormap_float = np.zeros((256, 3), np.float) 

for i in range(0, 256, 1):
    colormap_float[i, 0] = cm.jet(i)[0]
    colormap_float[i, 1] = cm.jet(i)[1]
    colormap_float[i, 2] = cm.jet(i)[2] 
        
    colormap_int[i, 0] = np.int_(np.round(cm.jet(i)[0] * 255.0))
    colormap_int[i, 1] = np.int_(np.round(cm.jet(i)[1] * 255.0))
    colormap_int[i, 2] = np.int_(np.round(cm.jet(i)[2] * 255.0)) 

C:\Users\win\AppData\Local\Temp/ipykernel_19420/519596690.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  colormap_float = np.zeros((256, 3), np.float)


In [7]:
datacube_1 = pd.read_csv(r'./Data/datacubes/datacube_1.csv', index_col=0)

In [8]:
# 读取原始图像
img_path = r'./Data/images/室内采集图像.JPG'
# img_path = r'./Data/images/手机采集图像.JPG'
reference_img_path = r'./Data/images/基准图像.JPG'
source_img = io.imread(img_path)
reference = io.imread(reference_img_path)
# 从多维数据集中读取图像
image_resized = np.array(ast.literal_eval(datacube_1.at[0, 'image_resized']))
Final_Spike_ROI_1 = np.array(ast.literal_eval(datacube_1.at[0, 'spike_mask']))

In [9]:
# 读取最佳分割比率和像素→毫米的转换关系
best_ratio = datacube_1.loc[0, 'best_ratio']
pm_ratio = datacube_1.loc[0, 'pm_ratio']

In [10]:
# 超像素分割原始图像
matched = match_histograms(source_img, reference, multichannel=True)
gray_image = matched[:,:,0] * resize(Final_Spike_ROI_1, (source_img.shape[0], source_img.shape[1]))
color_image = gray2color(gray_image, colormap_int)
segments_fz = felzenszwalb(color_image, scale=10, sigma=0.05, min_size=int(np.sum(resize(Final_Spike_ROI_1, (source_img.shape[0], source_img.shape[1])) > 0)/best_ratio))
grain_props = measure.regionprops_table(segments_fz, color.rgb2gray(source_img), 
                                        properties=['label', 'bbox', 'major_axis_length', 'minor_axis_length', 'area',  
                                                   'extent', 'solidity', 'image', 'intensity_image', 'eccentricity',
                                                    'perimeter_crofton'], extra_properties=(length,), 
                                        cache=True)
grain_data = pd.DataFrame(grain_props)

In [11]:
# 对超像素分割结果分类
for i in (grain_data.index):
    min_row = grain_data.loc[i,'bbox-0']
    min_col = grain_data.loc[i,'bbox-1']
    max_row = grain_data.loc[i,'bbox-2']
    max_col = grain_data.loc[i,'bbox-3']
    mask = grain_data.loc[i,'image']
    big_grain = (color.gray2rgb(mask)*source_img[min_row:max_row, min_col: max_col]).astype('int')
    big_grain = Image.fromarray(big_grain.astype('uint8'), mode='RGB')
    big_grain = tran(big_grain).cuda()
    big_grain.unsqueeze_(dim=0)
    out = model(big_grain)
    pred = torch.max(out, 1)[1][0].item()
    grain_data.at[i, 'pred'] = pred

In [12]:
# 统计穗粒相关性状
intact_grain = grain_data[grain_data['pred'] == 1]
intact_grain.reset_index(drop=True, inplace=True)
intact_grain['width'] = intact_grain['length'] / (intact_grain['major_axis_length'] / intact_grain['minor_axis_length'])# 粒宽
# 估算穗粒数
not_intact = grain_data[grain_data['pred'] == 0]
num = 0
for i in not_intact.index:
    num = num + round_new(not_intact.at[i, 'area'] / intact_grain['area'].mean(), 3)
intact_num = len(intact_grain)
grain_num = num + intact_num
# 长宽比均值
len_wid_mean = (intact_grain['major_axis_length'] / intact_grain['minor_axis_length']).mean()
# 长均值
length_mean = intact_grain['length'].mean() * pm_ratio 
# 宽均值
width_mean = intact_grain['width'].mean() * pm_ratio
# 面积均值
area_mean = intact_grain['area'].mean() * pow(pm_ratio,2)

# 统计粒色性状
img_exr = 1.4*matched[:,:,0] - matched[:,:,2] # 超红通道
for i in intact_grain.index:
    min_row = intact_grain.loc[i,'bbox-0']
    min_col = intact_grain.loc[i,'bbox-1']
    max_row = intact_grain.loc[i,'bbox-2']
    max_col = intact_grain.loc[i,'bbox-3']
    img_tmp_exr = img_exr[min_row:max_row, min_col:max_col] * intact_grain.at[i, 'image']
    exr_mean = img_tmp_exr.sum() / intact_grain.at[i, 'image'].sum()
    intact_grain.loc[i,'exr_mean'] = exr_mean
exr_mean_mean = intact_grain.loc[i,'exr_mean'].mean()
# 粒周长
perimeter_mean = intact_grain['perimeter_crofton'].mean() * pm_ratio
# 圆形度
roundness_mean = (4*np.pi*area_mean) / pow(perimeter_mean, 2)

C:\Users\win\AppData\Local\Temp/ipykernel_19420/110952075.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  intact_grain['width'] = intact_grain['length'] / (intact_grain['major_axis_length'] / intact_grain['minor_axis_length'])# 粒宽
C:\Users\win\anaconda3\envs\skimage18\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\win\anaconda3\envs\skimage18\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarn

In [13]:
# 输出性状分析结果
pandas_list = []
pandas_list.append([grain_num, best_ratio, intact_num, area_mean, len_wid_mean, 
                   length_mean, width_mean, perimeter_mean, roundness_mean, exr_mean_mean, 
                   pm_ratio])
result_dt = pd.DataFrame(pandas_list, columns=['穗粒数', '最佳分割比率', '单独穗粒数', 
                                              '粒面积均值', '长宽比均值', '粒长均值', 
                                               '粒宽均值', '粒周长均值', '粒圆形度均值',
                                              '粒色深浅', '毫米/像素比率'])
result_dt.to_csv(r'./Data/datacubes/result.csv', encoding='gbk')